<a href="https://pymt.readthedocs.io"><img style="float: right" src="images/pymt-logo-header-text.png"></a>

# Dynamically changing a running PyMT model

In this tutorial we will learn how to:
* Use the `update_until` method
* The model grid
* Change the input values of a model while it's running

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# Create and initialize a model

For this simulation, we'll be using the *Child* model with some non-default parameters.

In [ ]:
from pymt import MODELS

child = MODELS.Child()

Have a look under the *Parameters* help section (you may have to scroll down - it's the section after the citations). The *Parameters* section describes optional keywords that you can pass the the `setup` method. In the previous example we just used defaults. Below we'll see how to set input file parameters programmatically through keywords.

In [ ]:
config_file, config_dir = child.setup(
    grid_node_spacing=750.,
    grid_x_size=20000.,
    grid_y_size=40000.,
    run_duration=1e6,
)
child.initialize(config_file, dir=config_dir)

To begin with, we'll advance the model through 10 time steps.

In [ ]:
for t in tqdm(range(10)):
    child.update()

Update until some time in the future. Notice that, in this case, we update to a partial time step. Child is fine with this however some other models may not be. For models that can not update to times that are not full time steps, PyMT will advance to the next time step and interpolate values to the requested time.

In [ ]:
child.update_until(201.5)
child.time

Child offers different output variables but we get them in the same way as before.

In [ ]:
child.output_var_names

As before, we can get values of a variable with *get_value* (in whatever units we like).

In [ ]:
child.get_value("land_surface__elevation", units="cm")

We can query each input and output variable. PyMT attaches a dictionary to each component called `var` that provides information about each variable. For instance we can see that `"land_surface__elevation"` has units of meters, is an input and output variable, and is defined on the nodes of grid with id 0.

In [ ]:
child.var["land_surface__elevation"]

Notice that this variable is defined on grid with ID 0. We can get information about this grid through the `grid` attribute.

In [ ]:
child.grid[0]

We can also get grid information through method functions of the model. For example, the number of **nodes** that define each **face** of the grid.

In [ ]:
child.grid_nodes_per_face(0)

If we plot this variable, we can see the unsructured triangular grid that Child has decomposed its grid into.

In [ ]:
child.quick_plot('land_surface__elevation', edgecolors='k', vmin=-200, vmax=200, cmap='BrBG_r')

Child initializes it's elevations with random noise centered around 0. We would like instead to give it elevations that have some land and some sea. First we'll get the x and y coordinates for each node along with their elevations.

In [ ]:
x, y = child.grid_x(0), child.grid_y(0)
z = child.get_value('land_surface__elevation')

All nodes above `y=y_shore` will be land, and all nodes below `y=y_shore` will be sea.

In [ ]:
y_shore = 15000.
z[y < y_shore] -= 100
z[y >= y_shore] += 100

We now use the model's **set_value** method to change its current elevation values.

In [ ]:
child.set_value('land_surface__elevation', z)

Just to verify we set things up correctly, we'll create a plot.

In [ ]:
child.quick_plot('land_surface__elevation', edgecolors='k', vmin=-200, vmax=200, cmap='BrBG_r')

To get things going, we'll run the model for 5000 years and see what things look like.

In [ ]:
child.update_until(5000.)
child.quick_plot("land_surface__elevation", edgecolors="k", vmin=-200, vmax=200, cmap="BrBG_r")

## Exercise

We'll have some fun now by adding a simple uplift component. We'll run the component for another 5000 years but this time uplifting a corner of the grid by `dz_dt`. First, use the **get_value** method to create a new array of uplift values.

For this example, make the uplift zero everywhere except for *y>15km* and *x>10km* where it will be *0.02*.

In [ ]:
# Your code here

In [ ]:
x, y = child.grid_x(0), child.grid_y(0)

dz_dt = np.zeros_like(child.get_value("land_surface__elevation"))
dz_dt[(y > 15000.) & (x > 10000.)] = 0.02

## Exercise

Now with the uplift, we'll run the component for another 5000 years but this time uplifting a corner of the grid by `dz_dt` every time step.

In [ ]:
# Your code here

In [ ]:
# dz_dt = .02
now = child.time
times, dt = np.linspace(now, now + 5000., 50, retstep=True)

for time in tqdm(times):
    child.update_until(time)
    z = child.get_value('land_surface__elevation')
    
    z += dz_dt * dt
    # z[(y > 15000.) & (x > 10000.)] += dz_dt * dt
    child.set_value('land_surface__elevation', z)

In [ ]:
child.quick_plot("land_surface__elevation", edgecolors="k", vmin=-200, vmax=200, cmap="BrBG_r")

We now stop the uplift and run it for an additional 5000 years.

In [ ]:
for time in tqdm(np.linspace(child.time, child.time + 5000.)):
    child.update_until(time)
child.quick_plot("land_surface__elevation", edgecolors="k", vmin=-200, vmax=200, cmap="BrBG_r")